In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


# AUTOMATICALLY GENERATING CODE FOR CONTACT CORRELATIONS

In [ ]:
#get all of the possible 

targets = (dj.U("postsyn") & ta3p100.ContactTestSharedContact).proj(segment_id="postsyn") - ta3p100.SegmentExclude
info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
info

segment_pairs = info.fetch()
segment_pairs



In [6]:
import pandas as pd

#pull down the PrePost
# prepost_data = ta3p100.ContactPrePostTestReal.proj("presyn",
#         "postsyn","total_contact_conversion","total_contact_density","total_synapse_sizes_mean").fetch()

prepost_data = ta3p100.ContactPrePostTestReal.proj("presyn",
        "postsyn","total_contact_conversion","total_contact_density").fetch()


#put data in Dataframe
df = pd.DataFrame(prepost_data)

start_time = time.time()
#get one of the possible combinations from segment pairs
postsyn1 = 648518346341353019
postsyn2 = 648518346341352223

#get all of the rows with postsyn 1 and 2
df_1 = df[df["postsyn"]==postsyn1][:5]
df_2 = df[df["postsyn"]==postsyn2][1:]
#how you restrict a table by the values in the other table
df_1_common = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])
df_2_common = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])
#how to extract the data
#postsyns = restr_df["presyn"].to_numpy()
# df_1_common_conversion = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
# df_2_common_conversion = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
df_1_common_conversion = df_1_common["total_contact_conversion"].to_numpy()
df_2_common_conversion = df_2_common["total_contact_conversion"].to_numpy()

df_1_common_binary_conversion = np.copy(df_1_common_conversion)
df_2_common_binary_conversion = np.copy(df_2_common_conversion)


df_1_common_binary_conversion[df_1_common_binary_conversion>0] = 1.0
df_2_common_binary_conversion[df_2_common_binary_conversion>0] = 1.0

# print(df_1_common_binary_conversion)
# print(df_2_common_binary_conversion)

#postsyns = restr_df["presyn"].to_numpy()
#df_1_common_density = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
#df_2_common_density = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
df_1_common_density = df_1_common["total_contact_density"].to_numpy()
df_2_common_density = df_2_common["total_contact_density"].to_numpy()



#get the synaptic volume density
# df_1_common_synaptic_size = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_synapse_sizes_mean"].to_numpy()
# df_2_common_synaptic_size = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_synapse_sizes_mean"].to_numpy()
# df_1_common_synaptic_size_2 = df_1_common["total_synapse_sizes_mean"].to_numpy()
# df_1_common_synaptic_size_2 = df_2_common["total_synapse_sizes_mean"].to_numpy()

#calculate the coefficient
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
#start_time = time.time()

corr_conversion, p_value_conversion = pearsonr(df_1_common_conversion, df_2_common_conversion)
df_1_common_binary_conversion = df_1_common_binary_conversion.reshape(1,len(df_1_common_binary_conversion))
df_2_common_binary_conversion = df_2_common_binary_conversion.reshape(1,len(df_2_common_binary_conversion))
corr_density, p_value_density = pearsonr(df_1_common_density, df_2_common_density)
cos_lib = cosine_similarity(df_1_common_binary_conversion, df_2_common_binary_conversion)

# corr_synaptic_size, p_value_synaptic = pearsonr(df_1_common_synaptic_size, df_2_common_synaptic_size)
print(cos_lib[0][0])
print(corr_conversion)
print(corr_density)


# print(corr_synaptic_size)
print(f"Total time for correlation = {time.time()-start_time}")




0.8660254037844388
-0.029219759796989472
-0.6026168953755794
Total time for correlation = 0.010935783386230469


In [ ]:
pearsonr([1., 1., 1., 1.],[1., 1., 1., 1.])

In [ ]:
print(np.corrcoef([1., 1., 1., 1.],[1., 1., 1., 1.])[0, 1])

In [ ]:
#calculate the binary conversion rate
df_1_common_conversion[df_1_common_conversion>0] = 1
df_1_common_conversion

In [ ]:
x = np.array([0, 0.234, 0.99, 2.33, 2.5, 0, 0])
x[x>0] = 1
x

In [ ]:
y = np.copy(df_1_common_conversion)
y[y>0] = 1
y

In [ ]:
df_1_common_conversion

In [ ]:
x = np.array([1.0,1.0,1.0,1.0])
y = np.array([1.0,1.0,0.0,1.0])

In [ ]:
#try out the new correlation technique

In [ ]:
import scipy
scipy.stats.cov(x,y)

In [ ]:
x - y

In [ ]:
#// from http://skipperkongen.dk/2018/09/19/cosine-similarity-in-python/

import numpy as np
import time
from sklearn.metrics.pairwise import cosine_similarity

start_time = time.time()
# vectors
a = np.array([1.0,1.0,1.0,1.0])
b = np.array([1.0,1.0,0.0,1.0])

a = np.array([1.0,1.0,0.0,1.0])
b = np.array([1.0,1.0,0.0,1.0])
 
# # manually compute cosine similarity
# dot = np.dot(a, b)
# norma = np.linalg.norm(a)
# normb = np.linalg.norm(b)
# cos = dot / (norma * normb)


# # use library, operates on sets of vectors
# aa = a.reshape(1,4)
# ba = b.reshape(1,4)
# cos_lib = cosine_similarity(aa, ba)
cos_lib = cosine_similarity(a, b)
cos_lib
print(f"Took total sec: {time.time()-start_time}")
